In [12]:
import os
import psycopg2

# Database connection
try:
    conn = psycopg2.connect("dbname=imgclass user=postgres password=fahim host=localhost")
    cursor = conn.cursor()
    
    label_mapping = {"cats": "cat", "dogs": "dog"}  
    base_path = r"C:\Users\user\Downloads\DogsCats\dataset"

    inserted_count = 0  # Debugging count

    for split in ["training_set", "test_set"]:
        for label in ["cats", "dogs"]:
            folder_path = os.path.join(base_path, split, label)

            # Debug: Print detected images
            image_count = len([img for img in os.listdir(folder_path) if img.lower().endswith((".jpg", ".jpeg", ".png"))])
            print(f"Found {image_count} images in {folder_path}")

            for image_name in os.listdir(folder_path):
                if not image_name.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue  # Skip non-image files
                
                # Normalize path (important to avoid duplicates)
                image_path = os.path.join(folder_path, image_name)
                image_path = os.path.normpath(image_path).replace("\\", "/")  # Normalize slashes

                # Check if image already exists in DB
                cursor.execute("SELECT 1 FROM images WHERE image_path = %s LIMIT 1", (image_path,))
                exists = cursor.fetchone()

                if not exists:  # Insert only if the image is not already in the database
                    cursor.execute("INSERT INTO images (image_path, label) VALUES (%s, %s)", 
                                   (image_path, label_mapping[label]))
                    inserted_count += 1  # Track insertions

    conn.commit()
    print(f"Total new images inserted: {inserted_count}")

except psycopg2.Error as db_error:
    print(f"Database error: {db_error}")
    conn.rollback()

finally:
    cursor.close()
    conn.close()


Found 1700 images in C:\Users\user\Downloads\DogsCats\dataset\training_set\cats
Found 1700 images in C:\Users\user\Downloads\DogsCats\dataset\training_set\dogs
Found 425 images in C:\Users\user\Downloads\DogsCats\dataset\test_set\cats
Found 426 images in C:\Users\user\Downloads\DogsCats\dataset\test_set\dogs
Total new images inserted: 0


In [14]:
import psycopg2
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Database Connection
conn = psycopg2.connect("dbname=imgclass user=postgres password=fahim host=localhost")
cursor = conn.cursor()

# Fetch image paths and labels from database
cursor.execute("SELECT image_path, label FROM images")
data = cursor.fetchall()

# Close the database connection
cursor.close()
conn.close()

# Convert labels to numerical values
label_mapping = {"cat": 0, "dog": 1}
image_paths = []
labels = []

for image_path, label in data:
    image_paths.append(image_path)
    labels.append(label_mapping[label])

# Convert labels to NumPy array
labels = np.array(labels)


In [15]:
# Define image size
IMG_SIZE = (128, 128)

# Load images from paths
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, IMG_SIZE)  # Resize image
            img = img / 255.0  # Normalize pixel values
            images.append(img)
        else:
            print(f"Warning: Could not read {path}")  # Debugging missing images
    return np.array(images)

# Load images
images = load_images(image_paths)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")


Training samples: 3400, Testing samples: 851


In [16]:
# Define CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification (cat vs dog)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Save model
model.save("cats_vs_dogs_db_model.h5")


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 73s 622ms/step - accuracy: 0.4977 - loss: 0.7484 - val_accuracy: 0.5241 - val_loss: 0.6890
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 63s 585ms/step - accuracy: 0.5613 - loss: 0.6824 - val_accuracy: 0.5934 - val_loss: 0.6863
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 82s 588ms/step - accuracy: 0.5796 - loss: 0.6757 - val_accuracy: 0.6651 - val_loss: 0.6468
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 64s 594ms/step - accuracy: 0.6455 - loss: 0.6408 - val_accuracy: 0.6804 - val_loss: 0.6104
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 79s 567ms/step - accuracy: 0.6907 - loss: 0.5807 - val_accuracy: 0.6898 - val_loss: 0.5852
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 61s 569ms/step - accuracy: 0.7308 - loss: 0.5167 - val_accuracy: 0.7333 - val_loss: 0.5604
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 62s 575ms/step - accuracy: 0.7912 - loss: 0.4432 - val_accuracy: 0.7156 - val_loss: 0.5958
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 73s 678ms/step - accuracy: 0.8330 - loss: 0

In [18]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 198ms/step - accuracy: 0.7206 - loss: 0.6863
Test Accuracy: 73.33%


In [19]:
# Define model name
model_name = "cats_vs_dogs_model.keras"

# Save the model
model.save(model_name)

# Print success message
print(f'Model "{model_name}" saved successfully in Keras format!')


Model "cats_vs_dogs_model.keras" saved successfully in Keras format!


In [20]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the trained model
model_name = "cats_vs_dogs_model.keras"
model = keras.models.load_model(model_name)

# Define image size (same as used during training)
IMG_SIZE = (128, 128)

# Start webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Press 'Space' to capture an image and test it, or 'Q' to quit.")

while True:
    ret, frame = cap.read()  # Capture frame
    if not ret:
        print("Error: Failed to capture image.")
        break

    cv2.imshow("Live Camera - Press Space to Capture", frame)  # Show live feed

    key = cv2.waitKey(1)
    if key == ord(' '):  # Spacebar to capture
        image = cv2.resize(frame, IMG_SIZE)  # Resize to match model input
        image = image / 255.0  # Normalize
        image = np.expand_dims(image, axis=0)  # Expand dimensions for model

        # Prediction
        prediction = model.predict(image)[0][0]
        label = "Dog" if prediction > 0.5 else "Cat"

        print(f"Predicted Label: {label}")

        # Show result
        cv2.putText(frame, f"Prediction: {label}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Captured Image", frame)
        cv2.waitKey(2000)  # Show result for 2 seconds

    elif key == ord('q'):  # Quit on 'q' key
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
print("Camera closed successfully.")


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Press 'Space' to capture an image and test it, or 'Q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 628ms/step
Predicted Label: Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Label: Cat
Camera closed successfully.


In [22]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tkinter import Tk, filedialog

# Load the trained model
model_name = "cats_vs_dogs_model.keras"
model = keras.models.load_model(model_name)

# Define image size (same as used during training)
IMG_SIZE = (128, 128)
CONFIDENCE_THRESHOLD = 0.7  # Set a confidence threshold for classification

def select_image():
    """Opens file dialog to select an image"""
    Tk().withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(title="Select an Image",
                                           filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    return file_path

def preprocess_image(image_path):
    """Loads and preprocesses the image for model prediction"""
    image = cv2.imread(image_path)  # Read image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, IMG_SIZE)  # Resize to match model input
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Expand dimensions for model
    return image

# Select an image file
image_path = select_image()

if image_path:
    print(f"Selected Image: {image_path}")

    # Preprocess and predict
    image = preprocess_image(image_path)
    prediction = model.predict(image)[0][0]

    # Apply thresholding for unknown category
    if prediction < 0.5 - (1 - CONFIDENCE_THRESHOLD) or prediction > 0.5 + (1 - CONFIDENCE_THRESHOLD):
        label = "Unknown Category"
    else:
        label = "Dog" if prediction > 0.5 else "Cat"

    print(f"Predicted Label: {label}")

    # Show the image with prediction
    image_display = cv2.imread(image_path)
    cv2.putText(image_display, f"Prediction: {label}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Uploaded Image", image_display)
    cv2.waitKey(3000)  # Display for 3 seconds
    cv2.destroyAllWindows()

else:
    print("No image selected.")


Selected Image: C:/Users/user/Downloads/1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
Predicted Label: Unknown Category


In [6]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect("dbname=imgclass user=postgres password=fahim host=localhost")
cursor = conn.cursor()

# Load image paths and labels
df = pd.read_sql("SELECT image_path, label FROM images", conn)

# Close the connection
cursor.close()
conn.close()


C:\Users\user\AppData\Local\Temp\ipykernel_14736\1857612298.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT image_path, label FROM images", conn)


In [7]:
import tensorflow as tf

# Convert labels to numerical format
label_map = {"cat": 0, "dog": 1}  # Encoding: cat = 0, dog = 1
df["label"] = df["label"].map(label_map)

# Load and preprocess images
def load_and_preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode JPEG image
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((df["image_path"].values, df["label"].values))
dataset = dataset.map(load_and_preprocess).batch(32).shuffle(1000)


In [8]:
# Split dataset into training and testing
train_size = int(0.8 * len(df))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (cat/dog)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,396,609 (169.36 MB)

 Trainable params: 44,396,609 (169.36 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
train_dataset = train_dataset.batch(8)  # Reduce batch size from default 32 to 8
test_dataset = test_dataset.batch(8)


In [11]:
history= model.fit(train_dataset, epochs=10, validation_data=test_dataset)


Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, None, 224, 224, 3), dtype=float32). Expected shape (None, 224, 224, 3), but input has incompatible shape (None, None, 224, 224, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, 224, 224, 3), dtype=float32)
  • training=True
  • mask=None